In [ ]:
# Change directory to the root so that relative path loads work correctly
import os

try:
    os.chdir(os.path.join(os.getcwd(), ".."))
    print(os.getcwd())
except:
    pass

In [ ]:
import sys

import numpy as np
import torch

from experiments.A_proof_of_constraint.main import build_model_and_optimizer, get_data
from experiments.A_proof_of_constraint.visualize import (
    plot_constraints,
    plot_loss,
    plot_model_predictions,
    plot_time,
)

In [ ]:
# Files to load
directory = "/global/u1/g/gelijerg/Projects/pyinsulate/results/checkpoints/"
files = [
    # constrained
    "proof-of-constraint_2019-07-25-14-51-08_40.pth",  # start
    #     'proof-of-constraint_2019-07-25-15-00-37_40.pth', # uniform
    #     'proof-of-constraint_2019-07-25-15-10-28_40.pth', # random
    # unconstrained
    "proof-of-constraint_2019-07-25-14-10-06_200.pth",  # start
    #     'proof-of-constraint_2019-07-25-14-10-19_200.pth', # uniform
    #     'proof-of-constraint_2019-07-25-14-10-33_200.pth', # random
]

In [ ]:
# Load files
checkpoints = [torch.load(f"{directory}/{f}") for f in files]
filenames = [os.path.splitext(f)[0] for f in files]

In [ ]:
# Do some plotting
for checkpoint, filename in zip(checkpoints, filenames):

    training_monitor, evaluation_train_monitor, evaluation_test_monitor = checkpoint[
        "monitors"
    ]

    print(checkpoint["configuration"])
    #     print(training_monitor)
    #     print(evaluation_test_monitor.mean_loss)
    plot_loss(
        [evaluation_train_monitor, evaluation_test_monitor],
        ["Training", "Testing"],
        f"training-loss_{filename}",
    )
    plot_loss(
        [evaluation_train_monitor, evaluation_test_monitor],
        ["Training", "Testing"],
        f"training-constrained-loss_{filename}",
        title="Constrained losses",
        constrained=True,
    )
    plot_constraints(
        [evaluation_train_monitor, evaluation_test_monitor],
        ["Training", "Testing"],
        f"training-constraint_{filename}",
    )

    plot_time(training_monitor, f"compute-time_{filename}")

    # Try to retrieve the model and optimizer for plotting
    model, opt = build_model_and_optimizer(checkpoint["configuration"])
    model.load_state_dict(checkpoint["model_state_dict"])
    # Retrieve the data and equation
    train_dl, test_dl, parameterization, equation = get_data(
        checkpoint["configuration"], return_equation=True
    )

    # Use the models to predict over all the data
    models = [model]
    inputs = list()
    outputs = list()
    is_training = list()
    predictions = [list() for _ in models]
    for xb, yb in train_dl:
        inputs.extend(xb.squeeze().detach().numpy())
        outputs.extend(yb.squeeze().detach().numpy())
        is_training.extend([True for _ in range(len(xb.squeeze()))])
        for i, model in enumerate(models):
            model.eval()
            predictions[i].extend(model(xb).squeeze().detach().numpy())
    for xb, yb in test_dl:
        inputs.extend(xb.squeeze().detach().numpy())
        outputs.extend(yb.squeeze().detach().numpy())
        is_training.extend([False for _ in range(len(xb.squeeze()))])
        for i, model in enumerate(models):
            model.eval()
            predictions[i].extend(model(xb).squeeze().detach().numpy())
    # sort
    idxs = np.argsort(inputs)
    inputs = np.array(inputs)[idxs]
    outputs = np.array(outputs)[idxs]
    is_training = np.array(is_training)[idxs]
    predictions = np.array(predictions)[:, idxs]

    plot_model_predictions(
        (inputs, outputs, is_training),
        predictions,
        ["Unconstrained model"],
        f"predictions_{filename}",
    )